Chapter 33
Exporting Data

In [1]:
%reset -f

In [2]:
import pandas as pd 


In [18]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/dirtydevil.txt'

df = pd.read_csv(url, skiprows = lambda num:num <34 or num ==35, sep='\t')


C:\Users\wgrom\AppData\Local\Temp\ipykernel_14376\1734486613.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, skiprows = lambda num:num <34 or num ==35, sep='\t')


In [4]:
def to_america_denver_time(df_, time_col, tz_col):
    return(df_
           .assign(**{tz_col: df_[tz_col].replace('MDT','MST7MDT')})
           .groupby(tz_col)
           [time_col]
           .transform(lambda s:pd.to_datetime(s)
                      .dt.tz_localize(s.name, ambiguous=True)
                      .dt.tz_convert('America/Denver'))
           )
    

def tweak_river3(df_):
    return(df_
           .assign(datetime=to_america_denver_time(df_, 'datetime',
                    'tz_cd'))
           .rename(columns={'144166_00060':'cfs',
                            '144167_00065': 'gage_height'})
           .set_index('datetime')
          )
    
    
dd = tweak_river3(df)

In [21]:
# how many groups are there?
df.dtypes

agency_cd           object
site_no              int64
datetime            object
tz_cd               object
144166_00060       float64
144166_00060_cd     object
144167_00065       float64
144167_00065_cd     object
dtype: object

In [26]:
df['tz_cd'].value_counts()

MDT    341843
MST    197462
Name: tz_cd, dtype: int64